# Understanding Hidden Markov Models (HMM)

## Introduction to HMM

A Hidden Markov Model (HMM) is a statistical model that represents systems that are assumed to be Markov processes with unobserved (hidden) states. HMMs are widely used in various applications, including speech recognition, bioinformatics, and natural language processing.

### Key Components of HMM

1. **Hidden States**: These are the states of the system that are not directly observable. In our example, we have three hidden states:
   - Charging
   - Navigating
   - Exploring

2. **Observable States**: These are the states that can be observed or measured. In this example, the observable states are:
   - Idle
   - Move
   - Scan

3. **Start Probabilities**: These represent the initial probabilities of each hidden state.

4. **Transition Matrix**: This matrix defines the probabilities of transitioning from one hidden state to another.

5. **Emission Probabilities**: This matrix defines the probabilities of observing each observable state from a hidden state.

The following Python code uses the `hmmlearn` library to create and fit a Hidden Markov Model.

In [5]:
# Install hmmlearn library
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.6 MB/s eta 0:00:00


In [6]:
# Import libraries
import numpy as np
from hmmlearn import hmm

In [7]:
# Hidden states
hidden_states = ["Charging", "Navigating", "Exploring"]
id2state = dict(zip(range(len(hidden_states)), hidden_states))

# Prior probabilities for each hidden state
start_probs = np.array([0.3, 0.4, 0.3])  # 30% Charging, 40% Navigating, 30% Exploring

# Transition matrix for hidden states
trans_mat = np.array([[0.1, 0.8, 0.1],  # From Charging:   10% remain in Charging, 80% transition to Navigating, 10% transition to Exploring
                      [0.2, 0.6, 0.2],  # From Navigating: 20% transition to Charging, 60% remain in Navigating, 20% transition to Exploring
                      [0.5, 0.1, 0.4]]) # From Exploring:  50% transition to Charging, 10% transition to Navigating, 40% remain in Exploring

# Observable states and their corresponding emission probabilities
observable_states = ["idle", "move", "scan"]
emission_probs = np.array([[0.33, 0.34, 0.33],  # Emissions for Charging:   33% Idle, 34% Move, 33% Scan
                           [0.2, 0.1, 0.7],     # Emissions for Navigating: 20% Idle, 10% Move, 70% Scan
                           [0.2, 0.36, 0.44]])  # Emissions for Exploring:  20% Idle, 36% Move, 44% Scan

# List of observation sequences
observations = [["idle", "scan", "move"],
                ["move", "scan", "idle"],
                ["move", "move", "scan"],
                ["idle", "move", "scan"],
                ["scan", "idle", "idle"]]


In [8]:
# Create a mapping from observable states to IDs
obs2id = dict(zip(observable_states, range(len(observable_states))))

In [9]:

# Convert observations to counts
def observation2counts(observation):
    ans = []
    for word, idx in obs2id.items():
        count = observation.count(word)
        ans.append(count)
    return ans

# Prepare the data for the model
X = []
for observation in observations:
    row = observation2counts(observation)
    X.append(row)

data = np.array(X, dtype=int)
lengths = [len(X)] * 5
sequences = np.tile(data, (5, 1))

In [10]:
# Create and fit the HMM model
model = hmm.MultinomialHMM(n_components=len(hidden_states), n_iter=50, init_params='', random_state=0)
model.n_features = len(observable_states)
model.startprob_ = start_probs
model.transmat_ = trans_mat
model.emissionprob_ = emission_probs
model.fit(sequences, lengths)

# Decode the predicted states
logprob, predicted_states = model.decode(sequences)

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [11]:
# Predicted states and learned probabilities
print("Predicted states:")
print([id2state[x] for x in predicted_states])

Predicted states:
['Exploring', 'Exploring', 'Exploring', 'Charging', 'Navigating', 'Exploring', 'Exploring', 'Exploring', 'Charging', 'Navigating', 'Exploring', 'Exploring', 'Exploring', 'Charging', 'Navigating', 'Exploring', 'Exploring', 'Exploring', 'Charging', 'Navigating', 'Exploring', 'Exploring', 'Exploring', 'Charging', 'Navigating']


In [12]:
# Create a new model and fit it (the model will automatically initialize the parameters based on the training data).
new_model = hmm.MultinomialHMM(n_components=len(hidden_states), n_iter=50, init_params='ste', random_state=0)
new_model.fit(sequences, lengths)
logprob, new_predicted_states = new_model.decode(sequences)

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [13]:
# New model predictions and learned probabilities
print("\nNew Model Predictions:")
print("Predicted states:")
print([id2state[x] for x in new_predicted_states])

print("Learned emission probabilities:")
print(new_model.emissionprob_)

print("Learned transition matrix:")
print(new_model.transmat_)


New Model Predictions:
Predicted states:
['Charging', 'Navigating', 'Exploring', 'Charging', 'Charging', 'Charging', 'Navigating', 'Exploring', 'Charging', 'Charging', 'Charging', 'Navigating', 'Exploring', 'Charging', 'Charging', 'Charging', 'Navigating', 'Exploring', 'Charging', 'Charging', 'Charging', 'Navigating', 'Exploring', 'Charging', 'Navigating']
Learned emission probabilities:
[[3.33344955e-01 3.33321712e-01 3.33333333e-01]
 [4.99994189e-01 1.66672478e-01 3.33333333e-01]
 [6.74368905e-19 6.66666667e-01 3.33333333e-01]]
Learned transition matrix:
[[3.48649259e-05 9.99965135e-01 3.30003196e-15]
 [6.02988631e-19 5.90315044e-30 1.00000000e+00]
 [1.00000000e+00 6.23452134e-41 7.93807755e-33]]
